In [38]:
!pip install yfinance
!pip install pandas_datareader

In [1]:
import warnings

import yfinance as yf
from pandas_datareader import data as pdr

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree
from sklearn.metrics import mean_squared_error,confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC

In [2]:
data = pd.read_csv('all-trans-5-4-2021.csv')

In [3]:
drops = ['asset_description','comment','ptr_link']
columns_list = data.columns.values.tolist() #list of all columns
for i in range(len(columns_list)): #columns in table
    if columns_list[i] in drops: #if one of the variables we don't care about
        data.pop(columns_list[i]) #remove it

In [4]:
data.drop(data[data['asset_type'] != "Stock"].index, inplace = True) #remove all non stocks from the list of assets
data.drop(data[data['type'] != "Purchase"].index, inplace = True) #remove all non purchases from the list of types

In [5]:
def splice_date(date):
    date_to_list = list(date)
    date_to_string = ""
    for i in range(len(date_to_list)):
        if date_to_list[i] == '/':
            date_to_list[i] = '-'
    for i in range(len(date_to_list)):
        date_to_string = date_to_string + date_to_list[i]
    return date_to_string

In [12]:
test = data[800:-500]

In [13]:
def get_timelines():
    
    warnings.filterwarnings('ignore')
    
    trans_prices = []
    two_month_changes = [] #lists of the changes over each time period
    four_month_changes = []
    eight_month_changes = []
    one_year_changes = []
    
    i = 0
    
    for r,c in test.iterrows(): #iterate over the cases
        if i % 100 == 0:
            print(str(i)+" entries have been analyzed.")
        date = splice_date(c['transaction_date']) #index into the transaction date variable and splice it to work within the api
        ticker = c['ticker'] #grab the ticker symbol
        try: 
            stock_data = pdr.get_data_yahoo(ticker, start=date)
            trans_price = round(stock_data.iloc[0,2],3)
            two_month_change = round(stock_data.iloc[60,5] - stock_data.iloc[0,2],3)
            four_month_change = round(stock_data.iloc[120,5] - stock_data.iloc[0,2],3)
            eight_month_change = round(stock_data.iloc[240,5] - stock_data.iloc[0,2],3)
            one_year_change = round(stock_data.iloc[365,5] - stock_data.iloc[0,2],3)
            trans_prices.append(trans_price)
            two_month_changes.append(two_month_change)
            four_month_changes.append(four_month_change)
            eight_month_changes.append(eight_month_change)
            one_year_changes.append(one_year_change)
        except:
            NA = 'NA'
            trans_prices.append(NA)
            two_month_changes.append(NA)
            four_month_changes.append(NA)
            eight_month_changes.append(NA)
            one_year_changes.append(NA)
        i+=1
        if i == test.shape[0]:
            break
    
    test['_transaction_price'] = trans_prices
    test['_60_days'] = two_month_changes
    test['_120_days'] = four_month_changes
    test['_240_days'] = eight_month_changes
    test['_365_days'] = one_year_changes
    
    
    test.drop(test[test['_transaction_price'] == "NA"].index, inplace = True) #remove all assets that don't have a value so they arent searched for in the future
    test.drop(test[test['_60_days'] == "NA"].index, inplace = True) #remove all assets that don't have a value so they arent searched for in the future
    test.drop(test[test['_120_days'] == "NA"].index, inplace = True) #remove all assets that don't have a value so they arent searched for in the future
    test.drop(test[test['_240_days'] == "NA"].index, inplace = True) #remove all assets that don't have a value so they arent searched for in the future
    test.drop(test[test['_365_days'] == "NA"].index, inplace = True) #remove all assets that don't have a value so they arent searched for in the future

In [ ]:
get_timelines()

0 entries have been analyzed.
100 entries have been analyzed.
200 entries have been analyzed.
300 entries have been analyzed.
400 entries have been analyzed.
500 entries have been analyzed.
600 entries have been analyzed.
700 entries have been analyzed.


In [12]:
test.head(20)

,transaction_date,owner,ticker,asset_type,type,amount,senator,transaction_price,60_days,120_days,240_days,365_days
6436,01/05/2016,Child,NFLX,Stock,Purchase,"$1,001 - $15,000",Sheldon Whitehouse,110.45,-4.75,-25.12,14.55,41.93
6437,01/14/2016,Self,MRK,Stock,Purchase,"$1,001 - $15,000",Sheldon Whitehouse,50.67,-3.055,0.318,1.975,7.882
6445,01/05/2016,Spouse,CMI,Stock,Purchase,"$1,001 - $15,000",William Cassidy,90.05,5.255,2.261,33.057,55.105
6446,01/05/2016,Spouse,AAPL,Stock,Purchase,"$1,001 - $15,000",William Cassidy,26.438,-0.862,-4.905,0.944,7.483
6464,01/06/2016,Self,UA,Stock,Purchase,"$50,001 - $100,000",John Hoeven,41.406,-7.266,-7.516,-23.296,-26.016
6699,08/07/2015,Joint,CVX,Stock,Purchase,"$1,001 - $15,000",Patrick J Toomey,84.71,-10.123,-15.962,1.04,11.273
6844,09/10/2015,Self,IVZ,Stock,Purchase,"$1,001 - $15,000",Gary C Peters,32.35,-6.706,-10.448,-8.472,-5.609
6845,09/10/2015,Self,ABBV,Stock,Purchase,"$1,001 - $15,000",Gary C Peters,59.36,-15.237,-15.749,-6.811,-9.685
6846,09/10/2015,Self,T,Stock,Purchase,"$1,001 - $15,000",Gary C Peters,32.77,-7.94,-4.726,-1.9,-0.588
6847,09/16/2015,Self,PFE,Stock,Purchase,"$1,001 - $15,000",Gary C Peters,31.29,-6.12,-7.925,-3.228,-3.466


In [10]:
test.to_csv('500_trans_with_labels.csv')